Using data scraped from Mountain Project, a website where users share rock climbing routes, I created a plotly choropleth map that show which states have the most routes, while also displaying the amount of certain types of routes (toprope, boulder, etc) and the number of Crags (climbing areas). 

This is my first foray into gathering my own data, and also my first side project. Web scraping is very messy! From the start, I have been aware of the inherent flaws of this project. Many climbing areas are secretive and exclusive, and won't be shared with the public, let alone posted on Mountain Project. I personally know of a couple crags near where I live that aren't on Mountain Project, and I'm certainly no 'insider'.

Furthermore, if you compare the displayed Total number of climbs with the sum of the individual types of climbs, you may find that these amounts don't match. This is because some routes are considered both Trad and Toprope, so they get counted twice.

Overall, this project was meant to get me familiar with gathering and cleaning my own data, while creating something that is interesting to look at, even if it is not 100% accurate to reality. The biggest surprise is Massachusetts having the most crags, at 110, while California only has 19. California definitely has more than 19 areas, and most of the MA crags have less than 100 routes, while each CA area is upwards 250, with Joshua Tree alone having 6000 posted routes while all of MA only has 4000. Again, it is important note that these are merely the routes shared on the internet. My go-to crag is listed at under 300 routes, but if you ask the climbers who have been going there for years, they could show you over 1000 routes.

***Since this was originally made, some state pages have changed their layouts, making the 'number of crags' measuse displayed in the final map to be incorrect. For example, North Carolina's page used to just list out each crag on the page, but now the crags are separated by 4 different regions, where the actual list of areas for that region is found by clicking on the link to those regions. 

Code found in the "Final" notebook here: https://github.com/cpud/climb-plotly

In [2]:
# Trying to get hyperlinks for all states so we can run the scraping script on all states.

# Imports and get the html data from MountainProject. 
import pandas as pd
import numpy as np
import string
import requests
from bs4 import BeautifulSoup

#This page contains the links to each state page. 
url = 'https://www.mountainproject.com/route-guide'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# Put all hyperlinks on the page into a list.
data = []
for link in soup.find_all('a', class_= 'text-truncate float-xs-left'):
    data.append(link.get('href'))
    
# List of US States that we'll use to remove unwanted hyperlinks.
# Mississippi and Nebrasaka don't have pages, which makes sense,
# considering how flat they are.
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New-Hampshire","New-Jersey","New-Mexico","New-York",
  "North-Carolina","North-Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode-Island","South-Carolina","South-Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West-Virginia","Wisconsin","Wyoming"]

# List of US States and their abbreviations for a dictionary.
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New-Hampshire': 'NH',
    'New-Jersey': 'NJ',
    'New-Mexico': 'NM',
    'New-York': 'NY',
    'North-Carolina': 'NC',
    'North-Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode-Island': 'RI',
    'South-Carolina': 'SC',
    'South-Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West-Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Make the states lowercase to match hyperlinks. 
states = [x.lower() for x in states]
us_state_abbrev = dict((k.lower(),v) for k,v in us_state_abbrev.items())

# Loop through each hyperlink and see if it contains a state as a subsrting.
# If it does, add the link to a list containing links for each state page. 
US = []
for datum in data:
    for state in states:
        if state in datum:
            US.append([state,datum])
        
# Remove duplicate links, Vermont Ice+Mixed, and Tennessee Wall (they are duplicates!)
# Check length. Should be 48. 
res = [] 
[res.append(x) for x in US if x not in res] 
# US=list(set(US))
res.remove(['tennessee','https://www.mountainproject.com/area/105851828/the-tennessee-wall'])
#res.remove(['vermont','https://www.mountainproject.com/area/107280521/vermont-ice-and-mixed'])
links = dict(res)

In [3]:
url = links['alabama']

In [4]:
page = requests.get(url)
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data)

In [5]:
# Trying to get number of crags on an individual state page.

def getData(url):

    page = requests.get(url)

    # Getting the webpage, creating a Response object.
    response = requests.get(url)
 
    # Extracting the source code of the page.
    data = response.text
 
    # Passing the source code to Beautiful Soup to create a BeautifulSoup object for it.
    # soup = BeautifulSoup(data, 'lxml')
    soup = BeautifulSoup(data)
 
    # Extracting all the <a> tags into a list. No ID or class or anything next to hyperlink in HTML
    titles = soup.findAll('a')
    climbs = soup.findAll('span',class_ ='text-warm')
 
    # Extracting text from the the <a> and put into list. 
    areas = [x.text for x in titles]
    climbs = [y.text for y in climbs ]
            
    return areas, climbs


# Define function to get location of a string that always occurs after end of list of crags.
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
 
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

# Define function to remove the unwanted text from the dataframe. Pages with slight differences
# in their layout needed the cutOff point to be manually entered.
def adjustDF(url,state):

    areas, climbs = getData(url)
    
    # Cleaning up areas DataFrame
    areas = pd.DataFrame(list(filter(None,areas)))
    areas.columns = ['Crags']
    areas = areas[~areas['Crags'].str.contains('\n')]
    # Actual list of climbing areas starts 1 after 'Climbing Area map', and ends 2 before it.
    places = getIndexes(areas,'Climbing Area Map')
    cleaned_areas = areas.loc[places[0][0] + 1:places[1][0] - 2]
    
    # Cleaning up number of climbs DataFrame
    climbs = pd.DataFrame(list(filter(None,climbs)))
    climbs.columns = ['Climbs']
    climbs['Climbs'] = [s.strip() for s in climbs['Climbs']]
    
    # Make DataFrame combining fields. The climbs scrape starts with the number of climbs for
    # the first area, meaning it will line up with the proper crag. 
    df = pd.DataFrame(list(zip(cleaned_areas['Crags'],climbs['Climbs'])))

    # Add state column
    df = df.assign(ST = us_state_abbrev[state])
    return df

In [6]:
# Make combined dataframe for all crags in all states using gathered hyperlinks.
def makeDataFrame(links):
    # Instantiate empty dataframe
    df = pd.DataFrame()
    #Loop through links.
    for sta, link in links.items():
        # Add state dataframes on top of each other.
        df = df.append(adjustDF(link,sta))
    return df

In [7]:
all_df = makeDataFrame(links)
#all_df.info

In [8]:
all_df.head()

,0,1,ST
0,BankHead Forest/ Thompson Creek Trail,2,AL
1,Bankhead Forest/Ship Rock,10,AL
2,bankhead forest/sipsey picnic area,1,AL
3,Cherokee Bluffs,32,AL
4,Chewacla State Park,18,AL


In [9]:
all_df.columns=['Crags', 'Climbs', 'ST']
# Remove Punctuation from Climbs
all_df['Climbs'] = [s.translate(str.maketrans('', '', string.punctuation)) for s in all_df['Climbs']]
# Convert Climbs values to integers
all_df['Climbs'] = [int(x) for x in all_df['Climbs']]
# Make dataframe to count number of crags per state. 
crag_counts = pd.DataFrame(all_df['ST'].value_counts())
crag_counts.columns = ['Crags']
# Make dataframe for aggregating climb counts at the State level.
climb_counts = all_df.drop(columns = ['Crags'])
climb_counts = climb_counts.set_index('ST')
climb_counts = climb_counts.sum(level = 'ST')
# Make aggregate dataFrame.
agg_df = climb_counts.join(crag_counts)

In [10]:
# Define function to get climb type data (Number of boulders, sport, trad, etc)
def climb_type_dataframe(link, state):
    url = link
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    climb_types = ['Trad','Sport','Toprope','Boulder','Ice','Aid','Mixed','Alpine']
    route_count = []
    route_type = []
    # Loop through climb types to create lists. Lists will be cleaned and combined
    # into a dataframe.
    for climb_type in climb_types:
        for route in soup.find_all('span', class_ = "lef-nav-" + climb_type):
            route_count.append(route.text)
            route_type.append(climb_type)
            
    # Clean up lists before putting them in dataframe. 
    route_count = [s.strip() for s in route_count]
    route_count = [s.replace(' /','') for s in route_count]
    route_count = [s.replace(',','') for s in route_count]
    route_count = [int(s) for s in route_count]
    # Create dataframe. 
    df = pd.DataFrame(data = route_count)
    df['type'] = route_type
    df['ST'] = us_state_abbrev[state]
    return df

In [11]:
# Make dataframe with climb_types.
climb_df = pd.DataFrame()
for sta, link in links.items():
    climb_df = climb_df.append(climb_type_dataframe(link,sta))

In [12]:
# Set index and name columns for climb_df, join with agg_df for plotting. 
climb_df = climb_df.set_index('ST')
climb_df.columns = ['num','type']
# Pivot table to aggregate climb type counts to a state level.
table = climb_df.pivot_table(values = 'num', columns = 'type', index = 'ST', aggfunc = np.sum)
agg_df = agg_df.join(table)

In [13]:
# Create 'text' column in agg_df. This will be what displays when mouse
# hovers over a state. 
agg_df['text'] = ['Crags: {}<br>Boulder: {}<br>Sport: {}<br>Toprope: {}<br>Trad: {}<br>Alpine: {}<br>Ice: {}'\
                    .format(agg_df['Crags'][i],agg_df['Boulder'][i],agg_df['Sport'][i],agg_df['Toprope'][i],agg_df['Trad'][i],\
                    agg_df['Alpine'][i], agg_df['Ice'][i])\
                    for i in range(0, len(agg_df['Crags']))]

In [23]:
# Imports for chart
#import chart_studio.plotly as py
import plotly.graph_objs as go 
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline import download_plotlyjs, plot, iplot
import plotly.express as px

In [24]:
data = dict(type='choropleth',
            colorscale = 'inferno',
            locations = list(agg_df.index.values),
            z = agg_df['Climbs'],
            locationmode = 'USA-states',
            text = agg_df['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of Climbs"}
            ) 

In [25]:
layout = dict(title = 'Number of Climbs By State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

In [26]:
choromap = go.Figure(data = [data],layout = layout)

In [27]:
iplot(choromap)

In [28]:
#agg_df.to_csv('agg_df.csv')
#agg_df.to_csv('climb_data.csv')
#all_df.to_csv('all_df.csv')
#climb_df.to_csv('climb_df.csv')

In [29]:
pivot_df = climb_df.groupby(by = ['ST','type']).sum()
pivot_df['STtype'] = pivot_df.index
pivot_df['State'] = pivot_df['STtype'].apply(lambda x: x[0])
pivot_df['Type'] = pivot_df['STtype'].apply(lambda x: x[1])
pivot_df.drop('STtype', axis = 1, inplace = True)
pivot_df.columns = ['Climbs', 'State', 'Type']

In [30]:
fig = px.choropleth(pivot_df, 
                    locations = 'State',
                    color="Climbs", 
                    animation_frame="Type",
                    color_continuous_scale="Inferno",
                    locationmode='USA-states',
                    scope="usa",
                    #range_color=(0, 20),
                    title='Climbs by State and Type',
                    #height=600
                   )

In [31]:
fig.show()